In [1]:
import json, requests
import re
import pandas as pd
import numpy as np
from pymongo import ASCENDING, DESCENDING
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()
from folium import Map, Marker, Icon, FeatureGroup, LayerControl, Choropleth
from folium.plugins import HeatMap
import math
from geofun import *

# Select video-game and design companies in San Francisco (SF)

In [2]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [3]:
companies=list(db.companies.find({'category_code':{'$in':[re.compile(r"game(?i)"),re.compile(r"design(?i)")]}},{"name":1, "offices":1, "_id":0}))
df = pd.DataFrame(list(companies))

/home/jorge/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Flags not at the start of the expression 'game(?i)'
  """Entry point for launching an IPython kernel.
/home/jorge/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Flags not at the start of the expression 'design(?i)'
  """Entry point for launching an IPython kernel.


In [4]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")

In [5]:
clean_offices = pd.concat([df[["name"]], offices], axis=1)

In [ ]:
clean_offices["geopoint"] = clean_offices.apply(transformToGeoPoint, axis=1)

In [7]:
clean_offices= clean_offices[clean_offices["city"] == "San Francisco"]

In [8]:
clean_offices =clean_offices.dropna(subset=["latitude", "longitude"], how='all')
clean_offices["latitude"].isnull().value_counts()

False    50
Name: latitude, dtype: int64

In [9]:
clean_offices.to_json("offices.json",orient="records")

# Search Starbucks, Vegan Resturants, Basketball Stadiums, Parks and NightClubs

In [11]:
center_SF = geocode("San Francisco, CA")
center_SF

{'type': 'Point', 'coordinates': [-122.45712, 37.78986]}

In [14]:
starbucks_SF = getQuery("starbucks")

In [15]:
vegan_SF = getId("4bf58dd8d48988d1d3941735", 6000)

In [16]:
basketball_SF= getId("4bf58dd8d48988d18b941735", 10000)

In [17]:
party_SF= getQuery("4bf58dd8d48988d11f941735", 20000)

In [18]:
parks_SF= getQuery("4d4b7105d754a06377d81259", 20000)

# Keep as a dictionary 

In [19]:
starbucks_SF_dict=[]
for i in list(range(len(starbucks_SF["response"]["groups"][0]["items"]))):
        
            starbucks_SF_dict.append({
        "name": starbucks_SF["response"]["groups"][0]["items"][i]["venue"]["name"],
        "state_code": starbucks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["state"],
        "country_code": starbucks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["country"],
        "latitude": starbucks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": starbucks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [20]:
vegan_SF_dict=[]
for i in list(range(len(vegan_SF["response"]["groups"][0]["items"]))):
        
            vegan_SF_dict.append({
        "name": vegan_SF["response"]["groups"][0]["items"][i]["venue"]["name"],
        "state_code": vegan_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["state"],
        "country_code": vegan_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["country"],
        "latitude": vegan_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": vegan_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [21]:
basketball_SF_dict=[]
for i in list(range(len(basketball_SF["response"]["groups"][0]["items"]))):
        
            basketball_SF_dict.append({
        "name": basketball_SF["response"]["groups"][0]["items"][i]["venue"]["name"],
        "state_code": basketball_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["state"],
        "country_code": basketball_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["country"],
        "latitude": basketball_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": basketball_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [22]:
party_SF_dict=[]
for i in list(range(len(party_SF["response"]["groups"][0]["items"]))):
        
            party_SF_dict.append({
        "name": party_SF["response"]["groups"][0]["items"][i]["venue"]["name"],
        "state_code": party_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["state"],
        "country_code": party_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["country"],
        "latitude": party_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": party_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

In [23]:
parks_SF_dict=[]
for i in list(range(len(parks_SF["response"]["groups"][0]["items"]))):
        
            parks_SF_dict.append({
        "name": parks_SF["response"]["groups"][0]["items"][i]["venue"]["name"],
        "state_code": parks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["state"],
        "country_code": parks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["country"],
        "latitude": parks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lat"],
        "longitude": parks_SF["response"]["groups"][0]["items"][i]["venue"]["location"]["lng"]
    })

# Add geopoint column and keep as a json

In [24]:
starbucks = pd.DataFrame(starbucks_SF_dict)
starbucks["geopoint"] = starbucks.apply(transformToGeoPoint, axis=1)
starbucks.to_json("starbucks.json",orient="records")

In [25]:
vegan_restaurants = pd.DataFrame(vegan_SF_dict)
vegan_restaurants["geopoint"] = vegan_restaurants.apply(transformToGeoPoint, axis=1)
vegan_restaurants.to_json("vegan.json",orient="records")

In [26]:
basketball = pd.DataFrame(basketball_SF_dict)
basketball["geopoint"] = basketball.apply(transformToGeoPoint, axis=1)
basketball.to_json("basketball.json",orient="records")

In [27]:
party = pd.DataFrame(party_SF_dict)
party["geopoint"] = party.apply(transformToGeoPoint, axis=1)
party.to_json("party.json",orient="records")

In [28]:
parks = pd.DataFrame(parks_SF_dict)
parks["geopoint"] = parks.apply(transformToGeoPoint, axis=1)
parks.to_json("parks.json",orient="records")

# Select offices with near Starbucks, Vegan Resturants, Basketball Stadiums, Parks and NightClubs

In [30]:
near_offices_starbucks=[]
for i in range(len(starbucks)):
    q=(db.offices.find(geoQueryNear(starbucks.iloc[i].geopoint, radius=100),{"_id":0,"company_id":0}))
    near_offices_starbucks.append(list(q))
near_offices_starbucks=[x for i in near_offices_starbucks for x in i]
offices_starbucks=pd.DataFrame(near_offices_starbucks)
offices_starbucks.to_json("offices_starbucks.json",orient="records")

In [31]:
near_offices_vegan=[]
for i in range(len(vegan_restaurants)):
    q=(db.offices_starbucks.find(geoQueryNear(vegan_restaurants.iloc[i].geopoint, radius=100),{"_id":0,"company_id":0}))
    near_offices_vegan.append(list(q))
near_offices_vegan=[x for i in near_offices_vegan for x in i]
offices_vegan=pd.DataFrame(near_offices_vegan)
offices_vegan = offices_vegan.drop_duplicates(subset=["latitude","longitude"], keep="first")
offices_vegan.to_json("offices_vegan.json",orient="records")

In [32]:
near_offices_party=[]
for i in range(len(party)):
    q=(db.offices_vegan.find(geoQueryNear(party.iloc[i].geopoint, radius=1000),{"_id":0,"company_id":0}))
    near_offices_party.append(list(q))
near_offices_party=[x for i in near_offices_party for x in i]
offices_party=pd.DataFrame(near_offices_party)
offices_party = offices_party.drop_duplicates(subset=["latitude","longitude"], keep="first")
offices_party.to_json("offices_party.json",orient="records")

In [33]:
near_offices_parks=[]
for i in range(len(parks)):
    q=(db.offices_party.find(geoQueryNear(parks.iloc[i].geopoint, radius=1000),{"_id":0,"company_id":0}))
    near_offices_parks.append(list(q))
near_offices_parks=[x for i in near_offices_parks for x in i]
offices_parks=pd.DataFrame(near_offices_parks)
offices_parks = offices_parks.drop_duplicates(subset=["latitude","longitude"], keep="first")
offices_parks.to_json("offices_parks.json",orient="records")

In [34]:
offices_all_requirements=[]
for i in range(len(basketball)):
    q=(db.offices_parks.find(geoQueryNear(basketball.iloc[i].geopoint, radius=1000),{"_id":0,"company_id":0}))
    offices_all_requirements.append(list(q))
offices_all_requirements=[x for i in offices_all_requirements for x in i]
select_offices=pd.DataFrame(offices_all_requirements)
select_offices = select_offices.drop_duplicates(subset=["latitude","longitude"], keep="first")
select_offices.to_json("select_offices.json",orient="records")

In [47]:
select_offices.head()

,name,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Lexy,Corporate Headquarters,589 Howard Street,Fifth Floor,94105,San Francisco,CA,USA,37.786906,-122.397672,"{'type': 'Point', 'coordinates': [-122.397672,..."
1,NaturalMotion,San Francisco,143 2nd Street,Fifth Floor,94105,San Francisco,CA,USA,37.787530,-122.399000,"{'type': 'Point', 'coordinates': [-122.3989996..."
2,hi5,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."
3,Serious Business,,580 Market Street,#600,94104,San Francisco,CA,USA,37.789321,-122.401362,"{'type': 'Point', 'coordinates': [-122.4013624..."
4,PlayFirst,HQ,160 Spear St,Suite 1300,94105,San Francisco,CA,USA,37.790346,-122.401850,"{'type': 'Point', 'coordinates': [-122.40185, ..."


# Select location near Starbucks, Vegan Resturants, Basketball Stadiums, Parks and NightClubs of that offices

In [52]:
near_starbucks=[]
for i in range(len(select_offices)):
    q=(db.starbucks.find(geoQueryNear(select_offices.iloc[i].geopoint, radius=100),{"_id":0,"company_id":0}))
    near_starbucks.append(list(q))
near_starbucks=[x for i in near_starbucks for x in i]
off_starbucks=pd.DataFrame(near_starbucks)
off_starbucks = off_starbucks.drop_duplicates(subset=["latitude","longitude"], keep="first")
off_starbucks

,name,state_code,country_code,latitude,longitude,geopoint
0,Starbucks,CA,United States,37.788792,-122.400710,"{'type': 'Point', 'coordinates': [-122.4007102..."
1,Starbucks,CA,United States,37.787902,-122.401144,"{'type': 'Point', 'coordinates': [-122.4011438..."
2,Starbucks,CA,United States,37.789594,-122.402032,"{'type': 'Point', 'coordinates': [-122.402032,..."
4,Starbucks,CA,United States,37.790070,-122.402083,"{'type': 'Point', 'coordinates': [-122.4020826..."
6,Starbucks,CA,United States,37.787360,-122.403180,"{'type': 'Point', 'coordinates': [-122.40318, ..."


In [51]:
near_vegan=[]
for i in range(len(select_offices)):
    q=(db.vegan.find(geoQueryNear(select_offices.iloc[i].geopoint, radius=100),{"_id":0,"company_id":0}))
    near_vegan.append(list(q))
near_vegan=[x for i in near_vegan for x in i]
off_vegan=pd.DataFrame(near_vegan)
off_vegan = off_vegan.drop_duplicates(subset=["latitude","longitude"], keep="first")
off_vegan

,name,state_code,country_code,latitude,longitude,geopoint
0,sweetgreen,CA,United States,37.787262,-122.398635,"{'type': 'Point', 'coordinates': [-122.3986346..."
2,Sunrise Deli,CA,United States,37.788611,-122.400532,"{'type': 'Point', 'coordinates': [-122.4005322..."
3,Dosa Brothers,CA,United States,37.789021,-122.401848,"{'type': 'Point', 'coordinates': [-122.4018479..."
4,WeTheTrillions.,CA,United States,37.790158,-122.402439,"{'type': 'Point', 'coordinates': [-122.402439,..."
6,The Plant Cafe Organic,CA,United States,37.791888,-122.402499,"{'type': 'Point', 'coordinates': [-122.4024987..."
7,Ginto Izakaya Japonaise,CA,United States,37.788313,-122.402916,"{'type': 'Point', 'coordinates': [-122.402916,..."


In [49]:
near_party=[]
for i in range(len(select_offices)):
    q=(db.party.find(geoQueryNear(select_offices.iloc[i].geopoint, radius=1000),{"_id":0,"company_id":0}))
    near_party.append(list(q))
near_party=[x for i in near_party for x in i]
off_party=pd.DataFrame(near_party)
off_party = off_party.drop_duplicates(subset=["latitude","longitude"], keep="first")
off_party

,name,state_code,country_code,latitude,longitude,geopoint
0,AMC Metreon 16,CA,United States,37.784654,-122.403189,"{'type': 'Point', 'coordinates': [-122.4031889..."


In [48]:
near_parks=[]
for i in range(len(select_offices)):
    q=(db.parks.find(geoQueryNear(select_offices.iloc[i].geopoint, radius=1000),{"_id":0,"company_id":0}))
    near_parks.append(list(q))
near_parks=[x for i in near_parks for x in i]
off_parks=pd.DataFrame(near_parks)
off_parks = off_parks.drop_duplicates(subset=["latitude","longitude"], keep="first")
off_parks

,name,state_code,country_code,latitude,longitude,geopoint
0,Law Office of Amie D. Miller San Francisco Imm...,CA,United States,37.792836,-122.401090,"{'type': 'Point', 'coordinates': [-122.4010901..."
1,Nordstrom,CA,United States,37.783970,-122.407054,"{'type': 'Point', 'coordinates': [-122.4070539..."
2,"Michael D. Ross, Attorney at Law",CA,United States,37.793983,-122.403341,"{'type': 'Point', 'coordinates': [-122.403341,..."
12,Big 4 Restaurant,CA,United States,37.791490,-122.412393,"{'type': 'Point', 'coordinates': [-122.4123929..."
23,D&A Cafe,CA,United States,37.798004,-122.408142,"{'type': 'Point', 'coordinates': [-122.4081420..."


In [46]:
near_basketball=[]
for i in range(len(select_offices)):
    q=(db.basketball.find(geoQueryNear(select_offices.iloc[3].geopoint, radius=1000),{"_id":0,"company_id":0}))
    near_basketball.append(list(q))
near_basketball=[x for i in near_basketball for x in i]
off_basketball=pd.DataFrame(near_basketball)
off_basketball = off_basketball.drop_duplicates(subset=["latitude","longitude"], keep="first")
off_basketball.head()

,name,state_code,country_code,latitude,longitude,geopoint
0,Equinox Sports Club San Francisco,CA,United States,37.786649,-122.404568,"{'type': 'Point', 'coordinates': [-122.4045675..."
1,YMCA,CA,United States,37.792651,-122.392083,"{'type': 'Point', 'coordinates': [-122.3920834..."


# Create a dataframe with all locations

In [40]:
final = pd.concat([select_offices, off_starbucks, off_vegan, off_basketball, off_party, off_parks])

In [45]:
final.head()

,name,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geopoint
0,Lexy,Corporate Headquarters,589 Howard Street,Fifth Floor,94105,San Francisco,CA,USA,37.786906,-122.397672,"{'type': 'Point', 'coordinates': [-122.397672,..."
1,NaturalMotion,San Francisco,143 2nd Street,Fifth Floor,94105,San Francisco,CA,USA,37.787530,-122.399000,"{'type': 'Point', 'coordinates': [-122.3989996..."
2,hi5,,55 Second Street,Suite 400,94105,San Francisco,CA,USA,37.788668,-122.400558,"{'type': 'Point', 'coordinates': [-122.400558,..."
3,Serious Business,,580 Market Street,#600,94104,San Francisco,CA,USA,37.789321,-122.401362,"{'type': 'Point', 'coordinates': [-122.4013624..."
4,PlayFirst,HQ,160 Spear St,Suite 1300,94105,San Francisco,CA,USA,37.790346,-122.401850,"{'type': 'Point', 'coordinates': [-122.40185, ..."


# Calculate mean location

In [53]:
x = 0.0
y = 0.0
z = 0.0

for i, coord in final.iterrows():
    latitude = math.radians(coord.latitude)
    longitude = math.radians(coord.longitude)

    x += math.cos(latitude) * math.cos(longitude)
    y += math.cos(latitude) * math.sin(longitude)
    z += math.sin(latitude)

total = len(final)

x = x / total
y = y / total
z = z / total

central_longitude = math.atan2(y, x)
central_square_root = math.sqrt(x * x + y * y)
central_latitude = math.atan2(z, central_square_root)

new_office = {
    'latitude': math.degrees(central_latitude),
    'longitude': math.degrees(central_longitude)
    }

In [54]:
new_office

{'latitude': 37.78931154002931, 'longitude': -122.40202634116092}

In [84]:
office_location = officeLocation("37.78931154002931,-122.40202634116092")

In [86]:
office_location = {"Street":office_location["coordinates"]["staddress"], "Number":office_location["coordinates"]["stnumber"]  }

In [87]:
office_location

{'Street': 'MONTGOMERY ST', 'Number': '44'}

# Create a map

In [89]:
m = Map(location=[37.779026,-122.419906],zoom_start=12)

for i, row in select_offices.iterrows():
    icon = Icon(color="red",prefix="fa",icon="gamepad") 
    
    Marker(location=row[["latitude","longitude"]],tooltip="VideoGames Offices",
            popup=row["name"],icon=icon).add_to(m)
        
for i, row in off_starbucks.iterrows():
    icon = Icon(color="blue",prefix="fa",icon="coffee")   
    
    Marker(location=row[["latitude","longitude"]],tooltip="Starbucks",
            popup=row["name"],icon=icon).add_to(m)
        
for i, row in off_party.iterrows():     
    icon = Icon(color="black",prefix="fa",icon="glass")  
    
    Marker(location=row[["latitude","longitude"]],tooltip="Night Club",
            popup=row["name"],icon=icon).add_to(m)
        
for i, row in off_vegan.iterrows():
    icon = Icon(color="green",prefix="fa",icon="cutlery")  
    
    Marker(location=row[["latitude","longitude"]],tooltip="Vegan Resturant",
            popup=row["name"],icon=icon).add_to(m)
        
for i, row in off_basketball.iterrows():
    icon = Icon(color="cadetblue",prefix="fa",icon="heartbeat")  
    
    Marker(location=row[["latitude","longitude"]],tooltip="Basketball Stadium",
            popup=row["name"],icon=icon).add_to(m)

for i, row in off_parks.iterrows():     
    icon = Icon(color="darkgreen",prefix="fa",icon="tree")    
    
    Marker(location=row[["latitude","longitude"]],tooltip="Park",
            popup=row["name"],icon=icon).add_to(m)
        
icon = Icon(color="beige",prefix="fa",icon="star")           
Marker(location=[37.78931154002931,-122.40202634116092],
       popup=office_location,tooltip="New Office",icon=icon).add_to(m)


m

In [90]:
m.save('index.html')